In [1]:
import sys

project_root = (
    "/media/andrew/E4AB-09ED/Dev/py/cayce"
    if sys.platform == "linux"
    else "F:/Dev/py/cayce"
)
if project_root not in sys.path:
    sys.path.append(project_root)

In [11]:
import datetime as dt

from cayce.cik import get_ticker_to_cik_map
from cayce.query import EdgarIndex
from cayce.xbrl import FinancialStatementsParser

In [9]:
# First, let's name our security and identify the CIK code for it
ticker = "TSLA"
cik_code = get_ticker_to_cik_map()[ticker]
print("Loading statements for", ticker, "(", cik_code, ")")

Loading statements for TSLA ( 1318605 )


In [10]:
# set up an index for Edgar data
index = EdgarIndex()

In [14]:
# query for 10-Ks and 8-Ks in the past year
search_results = index.search(start_date=dt.date.today() - dt.timedelta(days=365), end_date=dt.date.today(), ciks=cik_code, form_types='10-K')
search_results

2020-10-04 22:40:20,809 : query._download_index : INFO : Using cached file C:\Users\Andrew\AppData\Local\Temp\tmp0qewkgvu\2020-4-index.zip


,company,form_type,cik,date_filed,file_name
494874,"Tesla, Inc.",10-K,1318605,2020-02-13,edgar/data/1318605/0001564590-20-004475.txt


In [18]:
search_results.iloc[0, :]

company                                       Tesla, Inc.
form_type                                            10-K
cik                                               1318605
date_filed                            2020-02-13 00:00:00
file_name     edgar/data/1318605/0001564590-20-004475.txt
Name: 494874, dtype: object

In [19]:
# download the XBRL file locally
local_xbrl_file = index.download_xbrl(search_results.iloc[0, :].values)
local_xbrl_file

2020-10-04 22:44:45,246 : query.download_xbrl : INFO : Begin downloading Tesla, Inc. form 10-K for 2020-02-13
2020-10-04 22:44:46,638 : query.download_xbrl : INFO : Writing local cache file C:\Users\Andrew\AppData\Local\Temp\tmp0qewkgvu\Tesla_Inc__10-K_20200213.xml
C:\Users\Andrew\AppData\Local\Temp\tmp0qewkgvu\Tesla_Inc__10-K_20200213.xml


In [27]:
search_results.iloc[0, :].form_type

&#39;10-K&#39;

In [29]:
# specify where to write this data to disk
from tempfile import gettempdir
from os import path
local_csv = path.join(gettempdir(), f"{ticker}_{search_results.iloc[0, :].form_type}_{search_results.iloc[0, :].date_filed:%Y-%m-%d}.csv")

# parse the xbrl file
statement_parser = FinancialStatementsParser(ticker, local_xbrl_file)
statements_df = statement_parser.parse_all()

# save and print to console, depending on how you wish to proceed
statements_df.head(10)
print("Writing to", local_csv)
statements_df.to_csv(local_csv)

,period_start,period_end,attribute_name,attribute_value,currency,Ticker,Category
0,2019-01-01,2019-12-31,documenttype,10-K,None,TSLA,DEI
1,2019-01-01,2019-12-31,documentperiodenddate,2019-12-31,None,TSLA,DEI
2,2019-01-01,2019-12-31,entityregistrantname,"Tesla, Inc.",None,TSLA,DEI
3,2019-01-01,2019-12-31,tradingsymbol,TSLA,None,TSLA,DEI
4,2019-01-01,2019-12-31,securityexchangename,NASDAQ,None,TSLA,DEI
5,None,2020-02-07,entitycommonstocksharesoutstanding,1.81342e+08,U_XBRLISHARES,TSLA,DEI
6,2019-01-01,2019-12-31,revenues,2.4578e+10,U_ISO4217USD,TSLA,Income
7,2018-01-01,2018-12-31,revenues,2.1461e+10,U_ISO4217USD,TSLA,Income
8,2017-01-01,2017-12-31,revenues,1.1759e+10,U_ISO4217USD,TSLA,Income
9,2019-01-01,2019-03-31,revenues,4.541e+09,U_ISO4217USD,TSLA,Income
